# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fdd6ca9ff70>
├── 'a' --> tensor([[ 0.2406,  0.4692, -0.6627],
│                   [ 0.4707, -0.7689,  0.1866]])
└── 'x' --> <FastTreeValue 0x7fdd6ca9feb0>
    └── 'c' --> tensor([[-0.4983,  0.7586,  1.2614,  1.0815],
                        [-0.2098,  0.7423, -0.7890,  2.3382],
                        [ 0.6822,  2.6437,  0.1399,  0.2523]])

In [4]:
t.a

tensor([[ 0.2406,  0.4692, -0.6627],
        [ 0.4707, -0.7689,  0.1866]])

In [5]:
%timeit t.a

72.5 ns ± 1.12 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fdd6ca9ff70>
├── 'a' --> tensor([[-0.5499, -0.0228, -0.9702],
│                   [-0.9479,  0.3183,  0.1676]])
└── 'x' --> <FastTreeValue 0x7fdd6ca9feb0>
    └── 'c' --> tensor([[-0.4983,  0.7586,  1.2614,  1.0815],
                        [-0.2098,  0.7423, -0.7890,  2.3382],
                        [ 0.6822,  2.6437,  0.1399,  0.2523]])

In [7]:
%timeit t.a = new_value

78.2 ns ± 0.692 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.2406,  0.4692, -0.6627],
               [ 0.4707, -0.7689,  0.1866]]),
    x: Batch(
           c: tensor([[-0.4983,  0.7586,  1.2614,  1.0815],
                      [-0.2098,  0.7423, -0.7890,  2.3382],
                      [ 0.6822,  2.6437,  0.1399,  0.2523]]),
       ),
)

In [10]:
b.a

tensor([[ 0.2406,  0.4692, -0.6627],
        [ 0.4707, -0.7689,  0.1866]])

In [11]:
%timeit b.a

65.9 ns ± 0.885 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-2.2743,  1.0135, -1.6793],
               [-3.4497, -1.2641,  0.2475]]),
    x: Batch(
           c: tensor([[-0.4983,  0.7586,  1.2614,  1.0815],
                      [-0.2098,  0.7423, -0.7890,  2.3382],
                      [ 0.6822,  2.6437,  0.1399,  0.2523]]),
       ),
)

In [13]:
%timeit b.a = new_value

607 ns ± 9.86 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

994 ns ± 13.5 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

13.4 µs ± 188 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

292 µs ± 5.76 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

269 µs ± 7.17 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fdcbe9a7f70>
├── 'a' --> tensor([[[ 0.2406,  0.4692, -0.6627],
│                    [ 0.4707, -0.7689,  0.1866]],
│           
│                   [[ 0.2406,  0.4692, -0.6627],
│                    [ 0.4707, -0.7689,  0.1866]],
│           
│                   [[ 0.2406,  0.4692, -0.6627],
│                    [ 0.4707, -0.7689,  0.1866]],
│           
│                   [[ 0.2406,  0.4692, -0.6627],
│                    [ 0.4707, -0.7689,  0.1866]],
│           
│                   [[ 0.2406,  0.4692, -0.6627],
│                    [ 0.4707, -0.7689,  0.1866]],
│           
│                   [[ 0.2406,  0.4692, -0.6627],
│                    [ 0.4707, -0.7689,  0.1866]],
│           
│                   [[ 0.2406,  0.4692, -0.6627],
│                    [ 0.4707, -0.7689,  0.1866]],
│           
│                   [[ 0.2406,  0.4692, -0.6627],
│                    [ 0.4707, -0.7689,  0.1866]]])
└── 'x' --> <FastTreeValue 0x7fdd6ca66430>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

44.7 µs ± 962 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fdd6ca98b80>
├── 'a' --> tensor([[ 0.2406,  0.4692, -0.6627],
│                   [ 0.4707, -0.7689,  0.1866],
│                   [ 0.2406,  0.4692, -0.6627],
│                   [ 0.4707, -0.7689,  0.1866],
│                   [ 0.2406,  0.4692, -0.6627],
│                   [ 0.4707, -0.7689,  0.1866],
│                   [ 0.2406,  0.4692, -0.6627],
│                   [ 0.4707, -0.7689,  0.1866],
│                   [ 0.2406,  0.4692, -0.6627],
│                   [ 0.4707, -0.7689,  0.1866],
│                   [ 0.2406,  0.4692, -0.6627],
│                   [ 0.4707, -0.7689,  0.1866],
│                   [ 0.2406,  0.4692, -0.6627],
│                   [ 0.4707, -0.7689,  0.1866],
│                   [ 0.2406,  0.4692, -0.6627],
│                   [ 0.4707, -0.7689,  0.1866]])
└── 'x' --> <FastTreeValue 0x7fdcb6d6bf40>
    └── 'c' --> tensor([[-0.4983,  0.7586,  1.2614,  1.0815],
                        [-0.2098,  0.7423, -0.7890,  2.3382],
                 

In [23]:
%timeit t_cat(trees)

42.2 µs ± 785 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

78.5 µs ± 1.1 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 0.2406,  0.4692, -0.6627],
                [ 0.4707, -0.7689,  0.1866]],
       
               [[ 0.2406,  0.4692, -0.6627],
                [ 0.4707, -0.7689,  0.1866]],
       
               [[ 0.2406,  0.4692, -0.6627],
                [ 0.4707, -0.7689,  0.1866]],
       
               [[ 0.2406,  0.4692, -0.6627],
                [ 0.4707, -0.7689,  0.1866]],
       
               [[ 0.2406,  0.4692, -0.6627],
                [ 0.4707, -0.7689,  0.1866]],
       
               [[ 0.2406,  0.4692, -0.6627],
                [ 0.4707, -0.7689,  0.1866]],
       
               [[ 0.2406,  0.4692, -0.6627],
                [ 0.4707, -0.7689,  0.1866]],
       
               [[ 0.2406,  0.4692, -0.6627],
                [ 0.4707, -0.7689,  0.1866]]]),
    x: Batch(
           c: tensor([[[-0.4983,  0.7586,  1.2614,  1.0815],
                       [-0.2098,  0.7423, -0.7890,  2.3382],
                       [ 0.6822,  2.6437,  0.1399,  0.2523]],
         

In [26]:
%timeit Batch.stack(batches)

105 µs ± 2.99 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 0.2406,  0.4692, -0.6627],
               [ 0.4707, -0.7689,  0.1866],
               [ 0.2406,  0.4692, -0.6627],
               [ 0.4707, -0.7689,  0.1866],
               [ 0.2406,  0.4692, -0.6627],
               [ 0.4707, -0.7689,  0.1866],
               [ 0.2406,  0.4692, -0.6627],
               [ 0.4707, -0.7689,  0.1866],
               [ 0.2406,  0.4692, -0.6627],
               [ 0.4707, -0.7689,  0.1866],
               [ 0.2406,  0.4692, -0.6627],
               [ 0.4707, -0.7689,  0.1866],
               [ 0.2406,  0.4692, -0.6627],
               [ 0.4707, -0.7689,  0.1866],
               [ 0.2406,  0.4692, -0.6627],
               [ 0.4707, -0.7689,  0.1866]]),
    x: Batch(
           c: tensor([[-0.4983,  0.7586,  1.2614,  1.0815],
                      [-0.2098,  0.7423, -0.7890,  2.3382],
                      [ 0.6822,  2.6437,  0.1399,  0.2523],
                      [-0.4983,  0.7586,  1.2614,  1.0815],
                      [-0.2098,  

In [28]:
%timeit Batch.cat(batches)

181 µs ± 1.74 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

557 µs ± 9.37 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
